In [1]:
%load_ext lineapy

In [2]:
import subprocess
import pathlib
import pandas as pd
import datetime
print('current UTC:', datetime.datetime.utcnow())

current UTC: 2022-10-13 04:10:08.497760


# Benchmark Model (`OptionPricing` library in R)

Here, we are using an R library called `OptionPricing` as the benchmark. Here are the steps:

* (optional) Modify the R source code if necessary.
    + Save the R source code as an artifact
* Use `subprocess` to run the R code
    + parameters is loaded within R session
    + R save the result into a local file
* Load the result generated by R and save it as an artifact

## Load R Source Code

In [3]:
Rbinary = '/home/mingjerli/miniconda3/envs/r_env/bin/Rscript'
R_script = 'optionpricing_call.r'
R_price_path = 'R_call_price.csv'

In [4]:
source_code = pathlib.Path(R_script).read_text()
print(source_code)
lineapy.save(source_code, 'R_source_code')

library(OptionPricing)
library(jsonlite)

config = fromJSON("../config/option_config.json")
european_call = BS_EC(K=config$K, r=config$r, sigma=config$v, T=config$T, S0=config$S)
print(european_call)
write.csv(european_call, file='R_call_price.csv')


LineaArtifact(name='R_source_code', _version=0)

## Execute R Code

and load result generated by R and save the result as an artifact

In [5]:
def run_R_test(Rbinary, R_script, R_price_path):
    p = subprocess.Popen([Rbinary, R_script])
    result = pd.read_csv(R_price_path, index_col=[0])
    return result.x.loc['price']

In [6]:
R_call_price = run_R_test(Rbinary, R_script, R_price_path)
lineapy.save(R_call_price, 'R_price')

      price       delta       gamma 
10.40353915  0.56552801  0.02262112 


LineaArtifact(name='R_price', _version=0)

In [7]:
R_call_price

10.4035391529966